# FIX Messages to Dictionary
https://bitnomial.com/docs/fix-dropcopy-protocol/

In [10]:
# Sample input string
input_str = "8=FIX.4.4 | 9=95 | 35=5 | 49=BTNL_PF | 56=fix_client | 34=25 | 52=20061124-15:59:50.524  | 58=NormalLogoutInitiatedByCounterparty | 10=054 |"

# Split the string by the delimiter "|"
pairs = input_str.split(" | ")

# Initialize an empty dictionary
result_dict = {}

# Iterate over each pair and split by "=" to get the key-value pairs
for pair in pairs:
    key, value = pair.split("=")
    result_dict[key] = value

# The resulting dictionary
print(result_dict)

# Iterate over the dictionary and clean the values
cleaned_data = {key: value.replace("|", "").strip() for key, value in result_dict.items()}

print(cleaned_data)

{'8': 'FIX.4.4', '9': '95', '35': '5', '49': 'BTNL_PF', '56': 'fix_client', '34': '25', '52': '20061124-15:59:50.524 ', '58': 'NormalLogoutInitiatedByCounterparty', '10': '054 |'}
{'8': 'FIX.4.4', '9': '95', '35': '5', '49': 'BTNL_PF', '56': 'fix_client', '34': '25', '52': '20061124-15:59:50.524', '58': 'NormalLogoutInitiatedByCounterparty', '10': '054'}


Install Pandas and SQLAlchemy (if not already installed):

In [ ]:
# Bash
#！pip install pandas sqlalchemy

In [15]:
import pandas as pd
from sqlalchemy import create_engine

def parse_line(line):
    """Parses a line of text into a dictionary of key-value pairs."""
    pairs = line.strip().split(" | ")
    result_dict = {}
    for pair in pairs:
        key, value = pair.split("=")
        result_dict[key] = value
        
    # Iterate over the dictionary and clean the values
    result_dict = {key: value.replace("|", "").strip() for key, value in result_dict.items()}
    return result_dict

def read_file_to_dict(file_path):
    """Reads a file and returns a list of dictionaries with the parsed key-value pairs."""
    messages = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Only process non-empty lines
                messages.append(parse_line(line))
    return messages

def store_to_dataframe(data):
    """Converts the list of dictionaries into a pandas DataFrame."""
    df = pd.DataFrame(data)
    return df

def save_dataframe_to_db(df, db_connection_string, table_name):
    """Saves the DataFrame to the specified database table."""
    engine = create_engine(db_connection_string)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Example usage
file_path = "fix_messages.txt"
data = read_file_to_dict(file_path)

# Store data in DataFrame
df = store_to_dataframe(data)
print(df)

         8   9 35          49          56  34                     52   98  \
0  FIX.4.4  88  A  fix_client     BTNL_PF   1  20061124-15:47:02.951    0   
1  FIX.4.4  57  0     BTNL_DC     BTNL_DC   2  20210525-16:59:02.564  NaN   
2  FIX.4.4  95  5     BTNL_PF  fix_client  25  20061124-15:59:50.524  NaN   

   108  553        554  141        10                                   58  
0   30   10  AUTHTOKEN    Y  CHECKSUM                                  NaN  
1  NaN  NaN        NaN  NaN       012                                  NaN  
2  NaN  NaN        NaN  NaN       054  NormalLogoutInitiatedByCounterparty  


In [16]:
import pandas as pd

# Convert the '52' column to datetime
df['52'] = pd.to_datetime(df['52'], format='%Y%m%d-%H:%M:%S.%f')

# Extract year, month, day, hour, minute, and second into new columns
df['Year'] = df['52'].dt.year
df['Month'] = df['52'].dt.month
df['Day'] = df['52'].dt.day
df['Hour'] = df['52'].dt.hour
df['Minute'] = df['52'].dt.minute
df['Second'] = df['52'].dt.second

print(df)

         8   9 35          49          56  34                      52   98  \
0  FIX.4.4  88  A  fix_client     BTNL_PF   1 2006-11-24 15:47:02.951    0   
1  FIX.4.4  57  0     BTNL_DC     BTNL_DC   2 2021-05-25 16:59:02.564  NaN   
2  FIX.4.4  95  5     BTNL_PF  fix_client  25 2006-11-24 15:59:50.524  NaN   

   108  553        554  141        10                                   58  \
0   30   10  AUTHTOKEN    Y  CHECKSUM                                  NaN   
1  NaN  NaN        NaN  NaN       012                                  NaN   
2  NaN  NaN        NaN  NaN       054  NormalLogoutInitiatedByCounterparty   

   Year  Month  Day  Hour  Minute  Second  
0  2006     11   24    15      47       2  
1  2021      5   25    16      59       2  
2  2006     11   24    15      59      50  


In [ ]:
# Save DataFrame to database
db_connection_string = 'sqlite:///your_database.db'  # Replace with your actual database connection string
table_name = 'your_table_name'
save_dataframe_to_db(df, db_connection_string, table_name)